In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# --------------------------------------------------------------------
# 📂 공용 라이브러리 로드 (Load Shared Library)
# --------------------------------------------------------------------
try:
    sys.path.append(os.path.abspath('../../01_Shared_Lib'))
    import constants as config
    import barh_plot as hbar
    print(f'📚 Shared Library Loaded: {config.__name__}')
except ImportError:
    print('⚠️ constants.py를 찾을 수 없습니다. (경로 확인 필요)')

# --------------------------------------------------------------------
# 🎨 한글 폰트 및 시각화 설정 (Korean Font & Visualization Settings)
# --------------------------------------------------------------------
if os.name == 'nt': # Windows
    plt.rc('font', family='Malgun Gothic')
elif os.name == 'posix': # Mac
    plt.rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지
plt.rcParams['figure.figsize'] = (10, 6)   # 기본 도화지 크기 설정

print('✅ Data Analysis Environment Initialized!')

In [ ]:
file_path = '../../00_Common_Data/Master_Sales_2025_Total.parquet'
df = pd.read_parquet(file_path)

df['ta_ymd'] = pd.to_datetime(df['ta_ymd'], format='%Y%m%d')

df['day'] = df['day'].map(config.DAY_KOR)
df['hour'] = df['hour'].map(config.TIME_SHORT_MAP)
df['age'] = df['age'].map(config.AGE_MAP)

target_cols = ['cty_rgn_no', 'admi_cty_no', 'card_tpbuz_cd', 'card_tpbuz_nm_1', 'card_tpbuz_nm_2', 'sex', 'age', 'day', 'hour']
for col in target_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')

df.info()

print('✅ Data Settings Environment Initialized!')

In [ ]:
import importlib
food_df = df[df['card_tpbuz_nm_1']=='음식']

group_food = food_df.groupby('card_tpbuz_nm_2',observed=True)['amt'].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(12,6))

importlib.reload(hbar)

hbar.draw_top_categories(group_food,"음식 총 매출",ax=ax)


In [ ]:
#남녀 관계에서 남자의 결제주도 가설.
food_df = df[df['card_tpbuz_nm_1'] == '음식']
food_df = food_df.groupby(['card_tpbuz_nm_2','sex'],observed=True)['amt'].sum().reset_index()
food_df

Male_food = food_df[food_df['sex']=='M']
Female_food = food_df[food_df['sex']=='F']


In [ ]:
# 파이 플롯
pie_df = df[df['card_tpbuz_nm_1']=='음식']
pie_df = pie_df.groupby('sex')['amt'].sum()

color = sns.color_palette('muted')
#단위 :억

sentence = []
for i ,(index, value) in enumerate(pie_df.items()):
    pct = (value/pie_df.sum()) * 100
    text = f"{index}\n{value/100000000:.0f}억 \n ({pct:.1f}%)"
    sentence.append(text)

plt.Figure(figsize=(6,4))
plt.title("성별 별 매출총액(%)", fontsize=15, weight='bold')
plt.pie(x=pie_df.values, labels=sentence, autopct=None, startangle=90, colors=color,
         explode=[0.05]*len(pie_df.index), labeldistance=0.5,
           wedgeprops={'width':0.6,'edgecolor': 'w', 'linewidth':3}, textprops={'fontsize':14, 'weight':'bold'})

#매출총액, 매출액평균, 매출건수, 매출건수평균 다 사진따야함.

In [ ]:
# 바 플롯
data = df[df['card_tpbuz_nm_1'] == '음식']

data = data.groupby(['card_tpbuz_nm_2', 'sex'], observed=True)['amt'].sum().reset_index()

# 정렬기준 만들기
order_data = data.groupby('card_tpbuz_nm_2', observed=True)['amt'].sum().sort_values(ascending=False)
my_order = order_data.index

#좀비처럼 살아있는 뒷 카테고리들
my_odrer = my_order.remove_unused_categories()


sex_order = ['M', 'F']

data['card_tpbuz_nm_2'] = data['card_tpbuz_nm_2'].cat.remove_unused_categories()
#정렬해야함.
#가독성 떨어짐 -> 범주형 떄문이였음.
#데이터 꾸미기
sns.barplot(data=data, x='amt', y='card_tpbuz_nm_2', hue='sex', order=my_order,  hue_order=['M','F'])


#다음엔 막대 위에 막대를 올려 퍼센트로 표시
#갑자기 다시 모든 업종이 나옴. 왜 ? 무엇이 그렇게 만든걸까 ?

In [ ]:
#박스플롯